# AI Service End-to-End Testing

This notebook tests the multi-stage vision analysis pipeline with **real pet images**.

**Pipeline stages:**
1. Content safety check (NSFW detection)
2. Species detection (dog/cat)
3. Breed classification (with crossbreed detection)
4. RAG enrichment (breed knowledge)
5. Contextual Ollama visual analysis

**Prerequisites:**
- All services running: `docker compose ps`
- AI service healthy: `docker exec ft_transcendence_ai_service curl http://localhost:3003/health`
- **Test images:** Place pet images in `test_data/images/` directory

In [ ]:
import requests
import json
import base64
import os
from pathlib import Path
from io import BytesIO
from PIL import Image
from IPython.display import display, HTML, Image as IPImage
import pandas as pd

# Configuration
AI_SERVICE_URL = "http://ai-service:3003"
TEST_IMAGES_DIR = Path("test_data/images")

print("✅ Imports loaded successfully")
print(f"📁 Test images directory: {TEST_IMAGES_DIR.absolute()}")

## 1. Load Available Test Images

Scan the test data directory for available pet images.

In [ ]:
# Supported image formats
IMAGE_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.webp', '.bmp'}

def find_test_images():
    """Find all image files in the test data directory."""
    if not TEST_IMAGES_DIR.exists():
        print(f"❌ Test images directory not found: {TEST_IMAGES_DIR.absolute()}")
        print("   Create it with: mkdir -p test_data/images")
        return []
    
    images = []
    for ext in IMAGE_EXTENSIONS:
        images.extend(TEST_IMAGES_DIR.glob(f'*{ext}'))
        images.extend(TEST_IMAGES_DIR.glob(f'*{ext.upper()}'))
    
    return sorted(images)

# Find available images
available_images = find_test_images()

if available_images:
    print(f"✅ Found {len(available_images)} test images:")
    for i, img_path in enumerate(available_images, 1):
        file_size = img_path.stat().st_size / 1024  # KB
        print(f"   {i}. {img_path.name} ({file_size:.1f} KB)")
else:
    print("⚠️  No test images found!")
    print(f"\n📝 Please add pet images to: {TEST_IMAGES_DIR.absolute()}")
    print("   Supported formats: JPG, JPEG, PNG, WEBP, BMP")
    print("\n   Example:")
    print("   cp /path/to/dog_photo.jpg test_data/images/")
    print("   cp /path/to/cat_photo.png test_data/images/")

## 2. Helper Functions

Functions to load images and convert to base64.

In [ ]:
def load_image_as_base64(image_path):
    """Load image file and convert to base64 data URI."""
    with open(image_path, 'rb') as f:
        image_data = f.read()
    
    # Detect format from file extension
    ext = image_path.suffix.lower().lstrip('.')
    if ext == 'jpg':
        ext = 'jpeg'
    
    encoded = base64.b64encode(image_data).decode()
    return f"data:image/{ext};base64,{encoded}"

def display_image(image_path, max_width=400):
    """Display image with optional max width."""
    img = Image.open(image_path)
    
    # Resize if too large
    if img.width > max_width:
        ratio = max_width / img.width
        new_size = (max_width, int(img.height * ratio))
        img = img.resize(new_size, Image.Resampling.LANCZOS)
    
    print(f"📸 Image: {image_path.name}")
    print(f"   Size: {img.width}x{img.height}")
    display(img)

print("✅ Helper functions loaded")

## 3. Select and Display Test Image

Choose an image to analyze.

In [ ]:
# Select first available image (or specify index)
if not available_images:
    print("❌ No images available. Please add images to test_data/images/ directory.")
    raise SystemExit("No test images found")

# Change this to test different images (0-indexed)
IMAGE_INDEX = 0

if IMAGE_INDEX >= len(available_images):
    print(f"⚠️  Image index {IMAGE_INDEX} out of range. Using first image.")
    IMAGE_INDEX = 0

selected_image_path = available_images[IMAGE_INDEX]
print(f"\n🎯 Selected image: {selected_image_path.name}\n")

# Display the selected image
display_image(selected_image_path)

# Convert to base64
test_image = load_image_as_base64(selected_image_path)
print(f"\n✅ Image encoded: {len(test_image)} bytes")

## 4. Test Full Vision Analysis Pipeline

Send the image through the complete multi-stage pipeline.

In [ ]:
print("🚀 Sending image to vision analysis pipeline...\n")

# Make request to vision analysis endpoint
try:
    response = requests.post(
        f"{AI_SERVICE_URL}/api/v1/vision/analyze",
        json={"image": test_image},
        timeout=60
    )
    
    print(f"✅ Response received")
    print(f"   Status Code: {response.status_code}")
    print(f"   Response Time: {response.elapsed.total_seconds():.2f}s")
    
    result = response.json()
    
except requests.exceptions.ConnectionError:
    print("❌ Connection failed. Make sure services are running:")
    print("   docker compose ps")
    raise
except requests.exceptions.Timeout:
    print("❌ Request timeout. Pipeline may be overloaded or Ollama is slow.")
    raise

# Show raw response for debugging
print(f"\n📄 Raw Response:")
print(json.dumps(result, indent=2))

## 5. Parse and Display Results

Extract and format the pipeline analysis results.

In [ ]:
def format_percentage(value):
    """Format float as percentage."""
    return f"{value * 100:.1f}%"

def display_results(result):
    """Display formatted analysis results."""
    if not result.get('success'):
        print("\n" + "="*60)
        print("❌ ANALYSIS FAILED")
        print("="*60)
        error = result.get('error', {})
        print(f"\nError Code: {error.get('code', 'UNKNOWN')}")
        print(f"Message: {error.get('message', 'No details available')}")
        return
    
    data = result['data']
    
    print("\n" + "="*60)
    print("🐕 VISION ANALYSIS RESULTS")
    print("="*60)
    
    # Species
    print(f"\n📊 Species Detection")
    print(f"   Detected: {data['species'].upper()}")
    
    # Breed Analysis
    breed = data['breed_analysis']
    print(f"\n🐾 Breed Analysis")
    print(f"   Primary Breed: {breed['primary_breed'].replace('_', ' ').title()}")
    print(f"   Confidence: {format_percentage(breed['confidence'])}")
    print(f"   Crossbreed: {'Yes' if breed['is_likely_crossbreed'] else 'No'}")
    
    # Breed probabilities
    if breed['breed_probabilities']:
        print(f"\n   📈 Top Breed Probabilities:")
        for i, bp in enumerate(breed['breed_probabilities'][:5], 1):
            breed_name = bp['breed'].replace('_', ' ').title()
            prob = format_percentage(bp['probability'])
            bar = '█' * int(bp['probability'] * 20)
            print(f"      {i}. {breed_name:25s} {prob:>6s} {bar}")
    
    # Crossbreed info
    if breed['crossbreed_analysis']:
        cross = breed['crossbreed_analysis']
        print(f"\n   🔀 Crossbreed Details:")
        print(f"      Common Name: {cross.get('common_name', 'Unknown')}")
        print(f"      Parent Breeds: {', '.join(cross['detected_breeds'])}")
        print(f"      Reasoning: {cross['confidence_reasoning']}")
    
    # Visual Description
    print(f"\n👁️  Visual Description")
    print(f"   {data['description']}")
    
    # Traits
    traits = data['traits']
    print(f"\n🎯 Traits")
    print(f"   Size: {traits['size'].capitalize()}")
    print(f"   Energy Level: {traits['energy_level'].capitalize()}")
    print(f"   Temperament: {traits['temperament']}")
    
    # Health Observations
    print(f"\n💊 Health Observations")
    if data['health_observations']:
        for i, obs in enumerate(data['health_observations'], 1):
            print(f"   {i}. {obs}")
    else:
        print("   ✓ No notable health observations")
    
    # RAG Enrichment
    if data.get('enriched_info'):
        enriched = data['enriched_info']
        print(f"\n📚 Knowledge Base Enrichment")
        
        if enriched.get('breed'):
            print(f"   Breed: {enriched['breed']}")
        
        if enriched.get('parent_breeds'):
            print(f"   Parent Breeds: {', '.join(enriched['parent_breeds'])}")
        
        if enriched.get('description'):
            print(f"\n   ℹ️  Breed Information:")
            desc = enriched['description']
            if len(desc) > 200:
                print(f"      {desc[:200]}...")
            else:
                print(f"      {desc}")
        
        if enriched.get('care_summary'):
            print(f"\n   🏥 Care Requirements:")
            print(f"      {enriched['care_summary'][:150]}..." if len(enriched['care_summary']) > 150 else f"      {enriched['care_summary']}")
        
        if enriched.get('health_info'):
            print(f"\n   ⚕️  Health Considerations:")
            print(f"      {enriched['health_info'][:150]}..." if len(enriched['health_info']) > 150 else f"      {enriched['health_info']}")
        
        if enriched.get('sources'):
            print(f"\n   📖 Sources: {', '.join(enriched['sources'])}")
    else:
        print(f"\n⚠️  RAG enrichment unavailable (graceful degradation)")
    
    print("\n" + "="*60)
    print(f"\n✅ Analysis complete for: {selected_image_path.name}")

# Display results
display_results(result)

## 6. Test Multiple Images

Analyze all available test images.

In [ ]:
def analyze_image(image_path, show_details=False):
    """Analyze a single image and return results."""
    try:
        image_base64 = load_image_as_base64(image_path)
        response = requests.post(
            f"{AI_SERVICE_URL}/api/v1/vision/analyze",
            json={"image": image_base64},
            timeout=60
        )
        
        result = response.json()
        
        if show_details:
            print(f"\n{'='*60}")
            print(f"📸 Image: {image_path.name}")
            display_results(result)
        
        return {
            'image': image_path.name,
            'status': response.status_code,
            'success': result.get('success', False),
            'species': result['data']['species'] if result.get('success') else None,
            'breed': result['data']['breed_analysis']['primary_breed'] if result.get('success') else None,
            'confidence': result['data']['breed_analysis']['confidence'] if result.get('success') else None,
            'crossbreed': result['data']['breed_analysis']['is_likely_crossbreed'] if result.get('success') else None,
            'latency': response.elapsed.total_seconds(),
            'error': result.get('error', {}).get('code') if not result.get('success') else None
        }
    except Exception as e:
        return {
            'image': image_path.name,
            'status': 'error',
            'success': False,
            'error': str(e)
        }

if len(available_images) > 1:
    print(f"\n🔄 Testing all {len(available_images)} images...\n")
    
    batch_results = []
    for i, img_path in enumerate(available_images, 1):
        print(f"[{i}/{len(available_images)}] Analyzing {img_path.name}...", end=' ')
        result = analyze_image(img_path, show_details=False)
        batch_results.append(result)
        
        if result['success']:
            print(f"✅ {result['species']} - {result['breed']} ({result['latency']:.2f}s)")
        else:
            print(f"❌ Failed: {result.get('error', 'Unknown')}")
    
    # Display summary table
    print(f"\n{'='*60}")
    print("📊 BATCH ANALYSIS SUMMARY")
    print(f"{'='*60}\n")
    
    df = pd.DataFrame(batch_results)
    
    # Format confidence as percentage
    if 'confidence' in df.columns:
        df['confidence'] = df['confidence'].apply(lambda x: f"{x*100:.1f}%" if pd.notna(x) else None)
    
    # Format latency
    if 'latency' in df.columns:
        df['latency'] = df['latency'].apply(lambda x: f"{x:.2f}s" if pd.notna(x) else None)
    
    display(df)
    
    # Statistics
    success_count = sum(1 for r in batch_results if r['success'])
    print(f"\n📈 Success Rate: {success_count}/{len(batch_results)} ({success_count/len(batch_results)*100:.1f}%)")
    
else:
    print("\nℹ️  Only one test image available. Add more images to test_data/images/ for batch testing.")

## 7. Service Health Checks

Verify all pipeline components are operational.

In [ ]:
print("Checking service health...\n")

services = [
    ("AI Service", "http://ai-service:3003/health"),
    ("Vision Route", "http://ai-service:3003/api/v1/vision/health"),
    ("Classification Service", "http://classification-service:3004/health"),
    ("Ollama", "http://ollama:11434/api/tags"),
]

health_results = []
for name, url in services:
    try:
        resp = requests.get(url, timeout=5)
        status = "✅ Healthy" if resp.status_code == 200 else "⚠️  Degraded"
        health_results.append({
            "Service": name,
            "Status": status,
            "Code": resp.status_code,
            "Response Time": f"{resp.elapsed.total_seconds():.3f}s"
        })
    except Exception as e:
        health_results.append({
            "Service": name,
            "Status": "❌ Down",
            "Code": "N/A",
            "Response Time": str(e)[:50]
        })

df = pd.DataFrame(health_results)
display(df)

# Check if all services healthy
all_healthy = all('✅' in r['Status'] for r in health_results)
if all_healthy:
    print("\n✅ All services operational")
else:
    print("\n⚠️  Some services are down or degraded")

## 8. Error Handling Test

Test pipeline rejection scenarios.

In [ ]:
print("Testing error handling...\n")

# Test with invalid base64
print("1. Testing invalid image format...")
try:
    invalid_response = requests.post(
        f"{AI_SERVICE_URL}/api/v1/vision/analyze",
        json={"image": "data:image/jpeg;base64,invalid_data"},
        timeout=30
    )
    print(f"   Status: {invalid_response.status_code}")
    error_result = invalid_response.json()
    if 'detail' in error_result:
        error_result = error_result['detail']
    print(f"   Error: {error_result.get('error', {}).get('code', 'N/A')}")
    print(f"   Message: {error_result.get('error', {}).get('message', 'N/A')}")
except Exception as e:
    print(f"   Exception: {e}")

print("\n✅ Error handling test complete")

## Summary

This notebook demonstrates:
- ✅ Loading real pet images from test data directory
- ✅ Multi-stage pipeline execution
- ✅ Species and breed classification
- ✅ Crossbreed detection
- ✅ RAG enrichment with breed knowledge
- ✅ Contextual visual analysis
- ✅ Batch testing multiple images
- ✅ Error handling and validation
- ✅ Service health monitoring

**Next Steps:**
- Add more test images to `test_data/images/`
- Validate crossbreed detection accuracy
- Monitor pipeline latency
- Test edge cases (blurry images, multiple pets, etc.)